# Retail Intelligence Platform: System & API Health Check

This notebook verifies the platform setup and API functionality step-by-step.

**Checklist:**
1. Check all dependencies installed without errors
2. Run all tests in `test_platform.py`
3. Start server and verify running at http://127.0.0.1:8000
4. Access API documentation at http://127.0.0.1:8000/docs
5. Generate sample data and verify success
6. Test file upload functionality
7. Verify all API endpoints respond correctly


In [ ]:
# Check All Dependencies Installed Without Errors
import subprocess
import sys

# List installed packages
installed = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze']).decode()
print('Installed packages:')
print(installed)

# Compare with requirements.txt
with open('requirements.txt', encoding='utf-8') as f:
    required = set(line.strip().split('==')[0] for line in f if line.strip() and not line.startswith('#'))
installed_pkgs = set(line.split('==')[0] for line in installed.splitlines())
missing = required - installed_pkgs
if missing:
    print(f"Missing packages: {missing}")
else:
    print("All required packages are installed.")


Installed packages:
aiofiles==23.2.1
annotated-types==0.7.0
anyio==3.7.1
asttokens==3.0.0
Babel==2.13.1
black==23.11.0
certifi==2025.7.14
charset-normalizer==3.4.2
click==8.2.1
cmdstanpy==1.2.5
colorama==0.4.6
comm==0.2.3
contourpy==1.3.2
convertdate==2.4.0
cycler==0.12.1
debugpy==1.8.15
decorator==5.2.1
ephem==4.2
et_xmlfile==2.0.0
exceptiongroup==1.3.0
executing==2.2.0
fastapi==0.104.1
flake8==6.1.0
fonttools==4.59.0
greenlet==3.2.3
h11==0.16.0
holidays==0.77
httpcore==1.0.9
httptools==0.6.4
httpx==0.25.2
idna==3.10
importlib_resources==6.5.2
iniconfig==2.1.0
ipykernel==6.30.0
ipython==8.37.0
jedi==0.19.2
Jinja2==3.1.2
joblib==1.5.1
jupyter_client==8.6.3
jupyter_core==5.8.1
kiwisolver==1.4.8
LunarCalendar==0.0.9
MarkupSafe==3.0.2
matplotlib==3.8.2
matplotlib-inline==0.1.7
mccabe==0.7.0
mypy_extensions==1.1.0
nest-asyncio==1.6.0
numpy==1.24.3
openpyxl==3.1.2
packaging==25.0
pandas==2.1.3
parso==0.8.4
pathspec==0.12.1
patsy==1.0.1
pillow==11.3.0
platformdirs==4.3.8
plotly==5.17.0
plugg

In [2]:
# Run All Tests in test_platform.py
import subprocess
result = subprocess.run([sys.executable, '-m', 'pytest', 'test_platform.py'], capture_output=True, text=True)
print(result.stdout)
if result.returncode == 0:
    print('All tests passed.')
else:
    print('Some tests failed. See output above.')


============================= test session starts =============================
platform win32 -- Python 3.10.9, pytest-7.4.3, pluggy-1.6.0
rootdir: d:\retail-intelligence
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=strict
collected 5 items

test_platform.py .....                                                   [100%]

============================== warnings summary ===============================
test_platform.py::test_api_server
  d:\retail-intelligence\retail_env\lib\site-packages\_pytest\python.py:198: PytestReturnNotNoneWarning: Expected None, but test_platform.py::test_api_server returned False, which will be an error in a future version of pytest.  Did you mean to use `assert` instead of `return`?
    warnings.warn(

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
======================== 5 passed, 1 warning in 5.08s =========================

All tests passed.


In [3]:
# Start Server and Verify Running at http://127.0.0.1:8000
import requests
try:
    response = requests.get('http://127.0.0.1:8000/health', timeout=5)
    print('Server running:', response.status_code == 200)
    print('Response:', response.json())
except Exception as e:
    print('Server not running or not accessible:', e)


Server running: True
Response: {'status': 'healthy', 'app_name': 'Retail Intelligence Platform', 'version': '1.0.0', 'environment': 'development'}


In [4]:
# Access API Documentation at http://127.0.0.1:8000/docs
try:
    response = requests.get('http://127.0.0.1:8000/docs', timeout=5)
    print('API docs accessible:', response.status_code == 200)
    print('Docs HTML preview:', response.text[:500])
except Exception as e:
    print('API docs not accessible:', e)


API docs accessible: True
Docs HTML preview: 
    <!DOCTYPE html>
    <html>
    <head>
    <link type="text/css" rel="stylesheet" href="https://cdn.jsdelivr.net/npm/swagger-ui-dist@5.9.0/swagger-ui.css">
    <link rel="shortcut icon" href="https://fastapi.tiangolo.com/img/favicon.png">
    <title>Retail Intelligence Platform - Swagger UI</title>
    </head>
    <body>
    <div id="swagger-ui">
    </div>
    <script src="https://cdn.jsdelivr.net/npm/swagger-ui-dist@5.9.0/swagger-ui-bundle.js"></script>
    <!-- `SwaggerUIBundle` is now av


In [5]:
# Generate Sample Data and Verify Success
try:
    from generate_sample_data import generate_sample_retail_data
    df, summary = generate_sample_retail_data()
    print(f"Generated {summary['total_records']} records")
    print(f"Date range: {summary['date_range']}")
    print(f"Total sales: ₹{summary['total_sales']:,.2f}")
except Exception as e:
    print('Sample data generation failed:', e)


✅ Sample data generated successfully!
📄 File: data/samples/indian_retail_sample.csv
📊 Records: 4,177
📅 Period: 2025-02-01 to 2025-07-31
💰 Total Sales: ₹527,841.64
🛍️ Products: 20
👥 Customers: 300
💳 Avg Transaction: ₹126.37
Generated 4177 records
Date range: 2025-02-01 to 2025-07-31
Total sales: ₹527,841.64


In [6]:
# Test File Upload Functionality
import os
try:
    sample_file = 'data/samples/retail_sample.csv'
    url = 'http://127.0.0.1:8000/upload/file'
    with open(sample_file, 'rb') as f:
        files = {'file': (os.path.basename(sample_file), f, 'text/csv')}
        response = requests.post(url, files=files)
    print('Upload response:', response.status_code)
    print('Response JSON:', response.json())
except Exception as e:
    print('File upload failed:', e)


Upload response: 404
Response JSON: {'error': 'Endpoint not found', 'status_code': 404}


In [36]:
# Verify All API Endpoints Respond Correctly
endpoints = [
    ('/api/v1/upload/list', 'get'),
    ('/api/v1/predict', 'post', {'file_id': 'sample', 'model_type': 'prophet', 'forecast_periods': 7}),
    ('/api/v1/models', 'get'),
    ('/api/v1/models/compare/sample', 'get'),
]
base_url = 'http://127.0.0.1:8000'
for ep in endpoints:
    url = base_url + ep[0]
    method = ep[1]
    print(f'Checking {url} ...')
    try:
        if method == 'get':
            response = requests.get(url, timeout=10)
        elif method == 'post':
            response = requests.post(url, json=ep[2], timeout=10)
        print('Status:', response.status_code)
        print('Response:', response.json() if 'application/json' in response.headers.get('content-type','') else response.text[:500])
    except Exception as e:
        print(f'Error accessing {url}:', e)


Checking http://127.0.0.1:8000/api/v1/upload/list ...
Status: 200
Response: {'success': True, 'files': [{'file_id': '26afd863-745b-4151-8854-c4bb266b9636', 'filename': 'retail_sample.csv', 'status': 'completed', 'uploaded_at': '2025-07-31T12:27:46.979281', 'completed_at': '2025-07-31T12:27:47.006245', 'file_size': 710}, {'file_id': '99696596-c613-4992-8299-bdac25475181', 'filename': 'retail_sample.csv', 'status': 'completed', 'uploaded_at': '2025-07-31T12:26:25.686658', 'completed_at': '2025-07-31T12:26:25.725872', 'file_size': 710}, {'file_id': '47d3cab4-33d2-4c47-87df-b2d7603c9ffa', 'filename': 'retail_sample.csv', 'status': 'completed', 'uploaded_at': '2025-07-31T12:26:01.788723', 'completed_at': '2025-07-31T12:26:01.807646', 'file_size': 710}, {'file_id': '4bbf39b9-a48f-4865-9144-28cb30516a83', 'filename': 'retail_sample.csv', 'status': 'completed', 'uploaded_at': '2025-07-31T12:23:11.958354', 'completed_at': '2025-07-31T12:23:11.986129', 'file_size': 710}], 'total_files': 4}
Check

In [13]:
# Upload a sample file and get file_id for forecasting
import os
sample_file = 'data/samples/retail_sample.csv'
url = 'http://127.0.0.1:8000/api/v1/upload'
file_id = None
try:
    with open(sample_file, 'rb') as f:
        files = {'file': (os.path.basename(sample_file), f, 'text/csv')}
        response = requests.post(url, files=files)
    print('Upload response:', response.status_code)
    print('Response JSON:', response.json())
    if response.status_code == 200 and 'file_id' in response.json():
        file_id = response.json()['file_id']
    else:
        print('Could not extract file_id from response.')
except Exception as e:
    print('File upload failed:', e)
file_id

Upload response: 202
Response JSON: {'success': True, 'data': {'file_id': '8596be8c-6345-4523-b84a-754ba5705749', 'filename': 'retail_sample.csv', 'status': 'processing'}, 'message': 'File uploaded successfully. Processing started.'}
Could not extract file_id from response.


In [14]:
# Extract file_id from upload response and poll for completion
import time
file_id = '8596be8c-6345-4523-b84a-754ba5705749'  # Extracted from previous response
status_url = f'http://127.0.0.1:8000/api/v1/upload/status/{file_id}'
for i in range(10):
    response = requests.get(status_url)
    print(f'Check {i+1}:', response.json())
    status = response.json().get('status')
    if status == 'completed':
        print('File processing completed.')
        break
    time.sleep(2)
else:
    print('File processing not completed after polling.')


Check 1: {'success': True, 'file_id': '8596be8c-6345-4523-b84a-754ba5705749', 'status': 'completed', 'progress': 100, 'results': {'success': True, 'data': 'Saved to processed_8596be8c-6345-4523-b84a-754ba5705749.csv', 'original_shape': [20, 5], 'processed_shape': [20, 8], 'column_analysis': {'date_column': 'date', 'sales_column': 'sales', 'quantity_column': 'quantity', 'product_column': 'product', 'customer_column': 'customer', 'numeric_columns': ['sales', 'quantity'], 'categorical_columns': ['date', 'product', 'customer'], 'detected_patterns': {'date': 'date', 'product': 'product', 'sales': 'sales', 'quantity': 'quantity', 'customer': 'customer'}}, 'validation_results': {'has_date_column': True, 'has_sales_column': True, 'has_sufficient_data': False, 'date_range_days': 9, 'missing_data_percentage': 0, 'seasonal_potential': False, 'trend_potential': True, 'issues': ['Less than 3 months of historical data', 'Insufficient data points: 20 (minimum 30 required)']}, 'quality_score': 68.0, '

In [6]:
# Verify All API Endpoints Respond Correctly with Real Data
import requests, time
base_url = 'http://127.0.0.1:8000'
sample_file = 'data/samples/retail_sample.csv'
files = {'file': open(sample_file, 'rb')}

# Upload file
response = requests.post(f'{base_url}/api/v1/upload', files=files)
try:
    resp_json = response.json()
except Exception:
    print('Upload failed. Response:', response.text)
    raise
file_id = resp_json.get('file_id') or (resp_json.get('data', {}) or {}).get('file_id')
print('Uploaded file_id:', file_id)
if not file_id:
    print('Upload failed. Response:', resp_json)
    raise Exception('Upload failed, no file_id returned')

# Poll for processing completion
list_url = f'{base_url}/api/v1/upload/list'
status = None
for _ in range(30):
    response = requests.get(list_url)
    result = response.json()
    status = next((f['status'] for f in result['files'] if f['file_id'] == file_id), None)
    print('Processing status:', status)
    if status == 'completed':
        break
    time.sleep(2)
if status != 'completed':
    print('File processing did not complete.')
    raise Exception('Processing failed')

# Train model (Prophet)
train_url = f'{base_url}/api/v1/train'
train_req = {'file_id': file_id, 'model_type': 'prophet'}
response = requests.post(train_url, json=train_req)
train_json = response.json()
job_id = train_json.get('job_id')
print('Training job_id:', job_id)

# Poll for training completion
status_url = f'{base_url}/api/v1/train/status/{job_id}'
train_status = None
for _ in range(30):
    response = requests.get(status_url)
    train_status = response.json()
    print('Training status:', train_status.get('status'))
    if train_status.get('status') == 'completed':
        break
    if train_status.get('status') == 'failed':
        print('Training failed:', train_status.get('error'))
        raise Exception('Training failed')
    time.sleep(2)
if train_status.get('status') != 'completed':
    print('Model training did not complete.')
    raise Exception('Training failed')

# Endpoints to verify
endpoints = [
    ('/api/v1/upload/list', 'get'),
    ('/api/v1/predict', 'post', {'file_id': file_id, 'model_type': 'prophet', 'forecast_periods': 7}),
    ('/api/v1/models', 'get'),
    (f'/api/v1/models/compare/{file_id}', 'get'),
]
for ep in endpoints:
    url = base_url + ep[0]
    method = ep[1]
    print(f'Checking {url} ...')
    try:
        if method == 'get':
            response = requests.get(url, timeout=10)
        elif method == 'post':
            response = requests.post(url, json=ep[2], timeout=10)
        print('Status:', response.status_code)
        print('Response:', response.json() if 'application/json' in response.headers.get('content-type','') else response.text[:500])
    except Exception as e:
        print(f'Error accessing {url}:', e)

Uploaded file_id: 26afd863-745b-4151-8854-c4bb266b9636
Processing status: completed
Training job_id: 9a145f12-c1d8-45b7-9144-08b9d3a13247
Training status: completed
Checking http://127.0.0.1:8000/api/v1/upload/list ...
Status: 200
Response: {'success': True, 'files': [{'file_id': '26afd863-745b-4151-8854-c4bb266b9636', 'filename': 'retail_sample.csv', 'status': 'completed', 'uploaded_at': '2025-07-31T12:27:46.979281', 'completed_at': '2025-07-31T12:27:47.006245', 'file_size': 710}, {'file_id': '99696596-c613-4992-8299-bdac25475181', 'filename': 'retail_sample.csv', 'status': 'completed', 'uploaded_at': '2025-07-31T12:26:25.686658', 'completed_at': '2025-07-31T12:26:25.725872', 'file_size': 710}, {'file_id': '47d3cab4-33d2-4c47-87df-b2d7603c9ffa', 'filename': 'retail_sample.csv', 'status': 'completed', 'uploaded_at': '2025-07-31T12:26:01.788723', 'completed_at': '2025-07-31T12:26:01.807646', 'file_size': 710}, {'file_id': '4bbf39b9-a48f-4865-9144-28cb30516a83', 'filename': 'retail_sampl

In [18]:
# Get a valid file_id from uploaded files and use for forecast endpoints
list_url = 'http://127.0.0.1:8000/api/v1/upload/list'
response = requests.get(list_url)
files = response.json().get('files', [])
file_id = None
if files:
    file_id = files[0]['file_id']
    print(f'Using file_id: {file_id}')
else:
    print('No uploaded files found.')

if file_id:
    endpoints = [
        ('/api/v1/predict', 'post', {'file_id': file_id, 'model_type': 'prophet', 'forecast_periods': 7}),
        (f'/api/v1/models/compare/{file_id}', 'get'),
    ]
    base_url = 'http://127.0.0.1:8000'
    for ep in endpoints:
        url = base_url + ep[0]
        method = ep[1]
        print(f'Checking {url} ...')
        try:
            if method == 'get':
                response = requests.get(url, timeout=30)
            elif method == 'post':
                response = requests.post(url, json=ep[2], timeout=30)
            print('Status:', response.status_code)
            print('Response:', response.json() if 'application/json' in response.headers.get('content-type','') else response.text[:500])
        except Exception as e:
            print(f'Error accessing {url}:', e)


No uploaded files found.


In [19]:
# Full workflow: upload, poll, train, poll, forecast
import requests, time
base_url = "http://localhost:8000/api/v1"
sample_file = "data/samples/retail_sample.csv"
files = {"file": open(sample_file, "rb")}

# Upload file
response = requests.post(f"{base_url}/upload", files=files)
resp_json = response.json()
file_id = resp_json.get("file_id")
print("Uploaded file_id:", file_id)

# Poll for processing completion
list_url = f"{base_url}/upload/list"
status = None
for _ in range(30):
    response = requests.get(list_url)
    result = response.json()
    status = next((f["status"] for f in result["files"] if f["file_id"] == file_id), None)
    print("Processing status:", status)
    if status == "completed":
        break
    time.sleep(2)
if status != "completed":
    print("File processing did not complete.")
    raise Exception("Processing failed")

# Train model (Prophet)
train_url = f"{base_url}/train"
train_req = {"file_id": file_id, "model_type": "prophet"}
response = requests.post(train_url, json=train_req)
train_json = response.json()
job_id = train_json.get("job_id")
print("Training job_id:", job_id)

# Poll for training completion
status_url = f"{base_url}/train/status/{job_id}"
train_status = None
for _ in range(30):
    response = requests.get(status_url)
    train_status = response.json()
    print("Training status:", train_status.get("status"))
    if train_status.get("status") == "completed":
        break
    if train_status.get("status") == "failed":
        print("Training failed:", train_status.get("error"))
        raise Exception("Training failed")
    time.sleep(2)
if train_status.get("status") != "completed":
    print("Model training did not complete.")
    raise Exception("Training failed")

# Forecast
predict_url = f"{base_url}/predict"
predict_req = {"file_id": file_id, "model_type": "prophet", "forecast_periods": 30}
response = requests.post(predict_url, json=predict_req)
predict_json = response.json()
if not predict_json.get("success"):
    print("Forecast error:", predict_json.get("message", predict_json.get("error")))
else:
    print("Forecast result:")
    print(predict_json["forecast"][:5])  # Show first 5 forecast rows

Upload response: 202
Uploaded file_id: 72b3ea82-f8bb-4fb9-9dab-b5be76da029f
Check 1: completed
File processing completed.
Checking http://127.0.0.1:8000/api/v1/predict ...
Error accessing http://127.0.0.1:8000/api/v1/predict: HTTPConnectionPool(host='127.0.0.1', port=8000): Read timed out. (read timeout=30)
Checking http://127.0.0.1:8000/api/v1/models/compare/72b3ea82-f8bb-4fb9-9dab-b5be76da029f ...
Error accessing http://127.0.0.1:8000/api/v1/predict: HTTPConnectionPool(host='127.0.0.1', port=8000): Read timed out. (read timeout=30)
Checking http://127.0.0.1:8000/api/v1/models/compare/72b3ea82-f8bb-4fb9-9dab-b5be76da029f ...
Error accessing http://127.0.0.1:8000/api/v1/models/compare/72b3ea82-f8bb-4fb9-9dab-b5be76da029f: HTTPConnectionPool(host='127.0.0.1', port=8000): Read timed out. (read timeout=30)
Error accessing http://127.0.0.1:8000/api/v1/models/compare/72b3ea82-f8bb-4fb9-9dab-b5be76da029f: HTTPConnectionPool(host='127.0.0.1', port=8000): Read timed out. (read timeout=30)


In [36]:
# Test /predict and /models/compare endpoints with provided file ID
import requests
base_url = 'http://127.0.0.1:8000/api/v1'
file_id = '48649579-6764-4fcc-939e-e9f3cca93514'

# Test /predict
predict_url = f'{base_url}/predict'
predict_req = {'file_id': file_id, 'model_type': 'prophet', 'forecast_periods': 7}
try:
    response = requests.post(predict_url, json=predict_req, timeout=15)
    print('Predict status:', response.status_code)
    if 'application/json' in response.headers.get('content-type',''):
        print('Predict response:', response.json())
    else:
        print('Predict response:', response.text[:500])
except Exception as e:
    print('Error accessing /predict:', e)

# Test /models/compare
compare_url = f'{base_url}/models/compare/{file_id}'
try:
    response = requests.get(compare_url, timeout=15)
    print('Compare status:', response.status_code)
    if 'application/json' in response.headers.get('content-type',''):
        print('Compare response:', response.json())
    else:
        print('Compare response:', response.text[:500])
except Exception as e:
    print('Error accessing /models/compare:', e)

Predict status: 500
Predict response: {'detail': 'Object of type Timestamp is not JSON serializable'}
Error accessing /models/compare: HTTPConnectionPool(host='127.0.0.1', port=8000): Read timed out. (read timeout=15)
Error accessing /models/compare: HTTPConnectionPool(host='127.0.0.1', port=8000): Read timed out. (read timeout=15)


In [8]:
# Practical API Test Client for Forecast Endpoints
import requests

class ForecastAPITester:
    def __init__(self, base_url="http://127.0.0.1:8000/api/v1"):
        self.base_url = base_url

    def get_file_list(self):
        url = f"{self.base_url}/upload/list"
        resp = requests.get(url)
        print("File List Status:", resp.status_code)
        print("File List Response:", resp.json())
        return resp.json().get("files", [])

    def get_file_info(self, file_id):
        files = self.get_file_list()
        for f in files:
            if f["file_id"] == file_id:
                print("File Info:", f)
                return f
        print("File not found:", file_id)
        return None

    def test_predict(self, file_id, model_type="prophet", forecast_periods=7, frequency="daily"):
        url = f"{self.base_url}/predict"
        payload = {
            "file_id": file_id,
            "model_type": model_type,
            "forecast_periods": forecast_periods,
            "frequency": frequency
        }
        resp = requests.post(url, json=payload)
        print("Predict Status:", resp.status_code)
        try:
            print("Predict Response:", resp.json())
        except Exception:
            print("Predict Response (raw):", resp.text[:500])

    def test_auto_forecast(self, file_id, forecast_periods=7):
        url = f"{self.base_url}/auto-forecast"
        payload = {
            "file_id": file_id,
            "forecast_periods": forecast_periods
        }
        resp = requests.post(url, json=payload)
        print("Auto-Forecast Status:", resp.status_code)
        try:
            print("Auto-Forecast Response:", resp.json())
        except Exception:
            print("Auto-Forecast Response (raw):", resp.text[:500])

    def test_compare(self, file_id):
        url = f"{self.base_url}/models/compare/{file_id}"
        resp = requests.get(url)
        print("Compare Status:", resp.status_code)
        try:
            print("Compare Response:", resp.json())
        except Exception:
            print("Compare Response (raw):", resp.text[:500])

# Usage Example:
tester = ForecastAPITester()
file_id = "48649579-6764-4fcc-939e-e9f3cca93514"  # Replace with your actual file_id
tester.get_file_info(file_id)
tester.test_predict(file_id)
tester.test_auto_forecast(file_id)
tester.test_compare(file_id)

File List Status: 200
File List Response: {'success': True, 'files': [], 'total_files': 0}
File not found: 48649579-6764-4fcc-939e-e9f3cca93514
Predict Status: 500
Predict Response: {'detail': ''}
Auto-Forecast Status: 422
Auto-Forecast Response: {'detail': [{'type': 'missing', 'loc': ['query', 'file_id'], 'msg': 'Field required', 'input': None, 'url': 'https://errors.pydantic.dev/2.5/v/missing'}]}
Compare Status: 404
Compare Response: {'error': 'Endpoint not found', 'status_code': 404}


In [9]:
# Automated workflow: upload, poll, extract file_id, and test all endpoints
import requests, time
base_url = 'http://127.0.0.1:8000/api/v1'
sample_file = 'data/samples/retail_sample.csv'
files = {'file': open(sample_file, 'rb')}

# Upload file
response = requests.post(f'{base_url}/upload', files=files)
resp_json = response.json()
file_id = resp_json.get('file_id') or (resp_json.get('data', {}) or {}).get('file_id')
print('Uploaded file_id:', file_id)
if not file_id:
    print('Upload failed. Response:', resp_json)
    raise Exception('Upload failed, no file_id returned')

# Poll for processing completion
list_url = f'{base_url}/upload/list'
status = None
for _ in range(30):
    response = requests.get(list_url)
    result = response.json()
    status = next((f['status'] for f in result['files'] if f['file_id'] == file_id), None)
    print('Processing status:', status)
    if status == 'completed':
        break
    time.sleep(2)
if status != 'completed':
    print('File processing did not complete.')
    raise Exception('Processing failed')

# Train model (Prophet)
train_url = f'{base_url}/train'
train_req = {'file_id': file_id, 'model_type': 'prophet'}
response = requests.post(train_url, json=train_req)
train_json = response.json()
job_id = train_json.get('job_id')
print('Training job_id:', job_id)

# Poll for training completion
status_url = f'{base_url}/train/status/{job_id}'
train_status = None
for _ in range(30):
    response = requests.get(status_url)
    train_status = response.json()
    print('Training status:', train_status.get('status'))
    if train_status.get('status') == 'completed':
        break
    if train_status.get('status') == 'failed':
        print('Training failed:', train_status.get('error'))
        raise Exception('Training failed')
    time.sleep(2)
if train_status.get('status') != 'completed':
    print('Model training did not complete.')
    raise Exception('Training failed')

# Test endpoints with valid file_id
print('\n--- Endpoint Tests ---')
# /predict
predict_url = f'{base_url}/predict'
predict_req = {'file_id': file_id, 'model_type': 'prophet', 'forecast_periods': 7}
response = requests.post(predict_url, json=predict_req)
print('Predict Status:', response.status_code)
print('Predict Response:', response.json() if 'application/json' in response.headers.get('content-type','') else response.text[:500])
# /auto-forecast
auto_url = f'{base_url}/auto-forecast?file_id={file_id}&forecast_periods=7'
response = requests.get(auto_url)
print('Auto-Forecast Status:', response.status_code)
print('Auto-Forecast Response:', response.json() if 'application/json' in response.headers.get('content-type','') else response.text[:500])
# /models/compare
compare_url = f'{base_url}/models/compare/{file_id}'
response = requests.get(compare_url)
print('Compare Status:', response.status_code)
print('Compare Response:', response.json() if 'application/json' in response.headers.get('content-type','') else response.text[:500])

Uploaded file_id: 2ac15af2-19f3-44f9-be28-4466a8f4d8b5
Processing status: completed
Training job_id: f532c5af-7293-49c7-a3dd-e9bf3ca7f7db
Training status: completed

--- Endpoint Tests ---
Predict Status: 200
Predict Response: {'success': True, 'forecast': [{'ds': '2025-06-11', 'yhat': 273.9526875833591, 'yhat_lower': 273.9526087608748, 'yhat_upper': 273.952756313504}, {'ds': '2025-06-12', 'yhat': 83.22731900187956, 'yhat_lower': 83.22724537302477, 'yhat_upper': 83.22738485418813}, {'ds': '2025-06-13', 'yhat': -20.186496013788748, 'yhat_lower': -20.186527097745326, 'yhat_upper': -20.186461713600902}, {'ds': '2025-06-14', 'yhat': -120.17341471865208, 'yhat_lower': -120.17369180462909, 'yhat_upper': -120.17310225949133}, {'ds': '2025-06-15', 'yhat': -241.61003797956243, 'yhat_lower': -241.61083911936743, 'yhat_upper': -241.6091697748569}, {'ds': '2025-06-16', 'yhat': -181.934412975195, 'yhat_lower': -181.9351989730114, 'yhat_upper': -181.93360141041822}, {'ds': '2025-06-17', 'yhat': -143

In [ ]:
# Test auto-forecast endpoint with GET and POST methods
import requests

def test_auto_forecast_methods(file_id):
    base_url = "http://127.0.0.1:8000/api/v1"
    url = f"{base_url}/auto-forecast?file_id={file_id}&forecast_periods=7"
    # Test GET
    resp = requests.get(url)
    print("GET Status:", resp.status_code)
    print("GET Response:", resp.json() if 'application/json' in resp.headers.get('content-type','') else resp.text[:500])
    # Test POST
    resp = requests.post(url)
    print("POST Status:", resp.status_code)
    print("POST Response:", resp.json() if 'application/json' in resp.headers.get('content-type','') else resp.text[:500])

# Usage: using the provided file_id
test_auto_forecast_methods("2ac15af2-19f3-44f9-be28-4466a8f4d8b5")

In [15]:
# Print the latest file_id from the upload list for endpoint testing
import requests
base_url = "http://127.0.0.1:8000/api/v1"
list_url = f"{base_url}/upload/list"
response = requests.get(list_url)
files = response.json().get("files", [])
if files:
    latest_file = files[0]
    print("Latest file_id:", latest_file["file_id"])
    print("File info:", latest_file)
else:
    print("No files found. Please upload a file first.")

Latest file_id: 2ac15af2-19f3-44f9-be28-4466a8f4d8b5
File info: {'file_id': '2ac15af2-19f3-44f9-be28-4466a8f4d8b5', 'filename': 'retail_sample.csv', 'status': 'completed', 'uploaded_at': '2025-07-31T11:36:58.782776', 'completed_at': '2025-07-31T11:36:58.865323', 'file_size': 710}


In [4]:
# Platform Health Check Summary
import subprocess, requests, os

print('Checking Python dependencies...')
try:
    result = subprocess.run(['pip', 'check'], capture_output=True, text=True)
    print('Dependencies:', 'OK' if result.returncode == 0 else result.stdout)
except Exception as e:
    print('Dependency check error:', e)

print('\nRunning tests...')
try:
    result = subprocess.run(['python', 'test_platfrom.py'], capture_output=True, text=True)
    print('Tests:', 'OK' if result.returncode == 0 else result.stdout)
except Exception as e:
    print('Test run error:', e)

print('\nChecking server status...')
try:
    resp = requests.get('http://127.0.0.1:8000')
    print('Server:', 'OK' if resp.status_code == 200 else f'Status {resp.status_code}')
except Exception as e:
    print('Server error:', e)

print('\nChecking API docs...')
try:
    resp = requests.get('http://127.0.0.1:8000/docs')
    print('API Docs:', 'OK' if resp.status_code == 200 else f'Status {resp.status_code}')
except Exception as e:
    print('Docs error:', e)

print('\nChecking sample data generation...')
try:
    result = subprocess.run(['python', 'generate_sample_data.py'], capture_output=True, text=True)
    print('Sample Data:', 'OK' if result.returncode == 0 else result.stdout)
except Exception as e:
    print('Sample data error:', e)

print('\nChecking file upload...')
try:
    sample_file = 'data/samples/retail_sample.csv'
    files = {'file': open(sample_file, 'rb')}
    resp = requests.post('http://127.0.0.1:8000/api/v1/upload', files=files)
    print('File Upload:', 'OK' if resp.status_code == 200 else f'Status {resp.status_code}')
except Exception as e:
    print('File upload error:', e)

print('\nChecking endpoint responses...')
endpoints = [
    '/api/v1/upload/list',
    '/api/v1/models',
]
for ep in endpoints:
    try:
        resp = requests.get(f'http://127.0.0.1:8000{ep}')
        print(f'{ep}:', 'OK' if resp.status_code == 200 else f'Status {resp.status_code}')
    except Exception as e:
        print(f'{ep} error:', e)

Checking Python dependencies...
Dependencies: OK

Running tests...
Tests: 

Checking server status...
Server: OK

Checking API docs...
API Docs: OK

Checking sample data generation...
Dependencies: OK

Running tests...
Tests: 

Checking server status...
Server: OK

Checking API docs...
API Docs: OK

Checking sample data generation...


Exception in thread Thread-25 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\priya\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "d:\retail-intelligence\retail_env\lib\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\priya\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\priya\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 1499, in _readerthread
    buffer.append(fh.read())
  File "C:\Users\priya\AppData\Local\Programs\Python\Python310\lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 187: character maps to <undefined>


Sample Data: OK

Checking file upload...
File Upload: Status 202

Checking endpoint responses...
/api/v1/upload/list: OK
/api/v1/models: OK
